# Tutorial for MOFF single HP1alpha dimer simulation.

In [ ]:
# load packages
import numpy as np
import pandas as pd
import sys
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit
import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import MOFFParser
from openabc.forcefields import MOFFMRGModel

# set simulation platform
platform_name = 'CPU'

We simulate a single HP1alpha dimer with MOFF force field. We start from the atomistic model (hp1a.pdb) to build the CA model and apply MOFF force field. For your reference, we also provide hp1a.itp file, which is the GROMACS topology file. 

In [ ]:
hp1alpha_dimer_parser = MOFFParser.from_atomistic_pdb('input-pdb/hp1a.pdb', 'hp1alpha_dimer_CA.pdb')

pdb = mdtraj.load_pdb('input-pdb/hp1a.pdb')
print('Show HP1alpha dimer all-atom structure.')
pdb.xyz -= np.mean(pdb.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(pdb)
view

In [ ]:
pdb = mdtraj.load_pdb('hp1alpha_dimer_CA.pdb')
print('Show HP1alpha dimer CG structure.')
pdb.xyz -= np.mean(pdb.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(pdb)
view

As shadow algorithm will find redundnat native pairs within disordered domains, we need to remove those redundant pairs. We only keep native pairs within CD, CSD, or between two CSDs. The CD domain is 17-72, and CSD domain is 115-176 (here indices start from 1), and there are 191 residues in each chain.

In [ ]:
old_native_pairs = hp1alpha_dimer_parser.native_pairs.copy()
new_native_pairs = pd.DataFrame(columns=old_native_pairs.columns)
cd1 = np.arange(16, 72)
csd1 = np.arange(114, 176)
n_atoms_per_hp1alpha_dimer = len(hp1alpha_dimer_parser.atoms.index)
print(f'There are {n_atoms_per_hp1alpha_dimer} CA atoms in each HP1alpha dimer.')
cd2 = cd1 + int(n_atoms_per_hp1alpha_dimer/2)
csd2 = csd1 + int(n_atoms_per_hp1alpha_dimer/2)
for i, row in old_native_pairs.iterrows():
    a1, a2 = int(row['a1']), int(row['a2'])
    if a1 > a2:
        a1, a2 = a2, a1
    flag1 = ((a1 in cd1) and (a2 in cd1)) or ((a1 in csd1) and (a2 in csd1))
    flag2 = ((a1 in cd2) and (a2 in cd2)) or ((a1 in csd2) and (a2 in csd2))
    flag3 = ((a1 in csd1) and (a2 in csd2))
    if flag1 or flag2 or flag3:
        new_native_pairs.loc[len(new_native_pairs.index)] = row
hp1alpha_dimer_parser.native_pairs = new_native_pairs
hp1alpha_dimer_parser.parse_exclusions() # update exclusions based on the new native pairs

We can read the bonded interactin parameters, which are pandas dataframes set as attributes of `hp1alpha_dimer_parser`. 

We print the first 5 rows of protein bonds, angles, dihedrals, and native pairs to take a look.

In [ ]:
print(hp1alpha_dimer_parser.protein_bonds.head())
print(hp1alpha_dimer_parser.protein_angles.head())
print(hp1alpha_dimer_parser.protein_dihedrals.head())
print(hp1alpha_dimer_parser.native_pairs.head())

Now we can do the simulation. We append `hp1alpha_dimer_parser` to `protein`, and object `protein` includes `protein_bonds`, `protein_angles`, `protein_dihedrals`, and `native_pairs` as attributes. Note we use stronger native pairs (epsilon = 6.0 instead of 3.0). We directly change the epsilon parameter in `protein.native_pairs`. 

In [ ]:
protein = MOFFMRGModel()
protein.append_mol(hp1alpha_dimer_parser)
protein.native_pairs.loc[:, 'epsilon'] = 6.0
top = app.PDBFile('hp1alpha_dimer_CA.pdb').getTopology()
protein.create_system(top, box_a=50, box_b=50, box_c=50)
salt_conc = 82*unit.millimolar
temperature = 300*unit.kelvin
protein.add_protein_bonds(force_group=1)
protein.add_protein_angles(force_group=2, verbose=False)
protein.add_protein_dihedrals(force_group=3)
protein.add_native_pairs(force_group=4)
protein.add_contacts(force_group=5)
protein.add_elec_switch(salt_conc, temperature, force_group=6)
protein.save_system('hp1alpha_dimer_CA_system.xml')
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBFile('hp1alpha_dimer_CA.pdb').getPositions()
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_dimer.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

In [ ]:
# view trajectory
traj = mdtraj.load_dcd('output_dimer.dcd', top='hp1alpha_dimer_CA.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

One useful setting is to apply rigid bodies. We use HP1alpha dimer as an example to show how to manipulate rigid body with our tool. The rigid body code is written by Peter Eastman, and we provide useful methods to help remove bonded interactions within the rigid body and make the simulation more efficient. The rigid body is realized by representing each rigid body with few real sites and certain restraints, while viewing the rest atoms as virtual sites.

In the following sessions, we rigidize each CD domain and two CSDs together (i.e. 3 rigid bodies in all) based on the native configuration. All the bonded interactions (bonds, angles, and dihedrals) within each rigid body will be removed to save computational resources. Additionally we do not need to add native pairs as they are all within the rigid bodies. 

In [ ]:
protein = MOFFMRGModel()
protein.append_mol(hp1alpha_dimer_parser)
top = app.PDBFile('hp1alpha_dimer_CA.pdb').getTopology()
protein.create_system(top, box_a=50, box_b=50, box_c=50)
init_coord = app.PDBFile('hp1alpha_dimer_CA.pdb').getPositions()
rigid_coord = init_coord # set rigid body coordinates
rigid_bodies = [cd1.tolist(), cd2.tolist(), csd1.tolist() + csd2.tolist()] # 3 rigid bodies
protein.set_rigid_bodies(rigid_coord, rigid_bodies)
salt_conc = 82*unit.millimolar
temperature = 300*unit.kelvin
protein.add_protein_bonds(force_group=1)
protein.add_protein_angles(force_group=2, verbose=False)
protein.add_protein_dihedrals(force_group=3)
protein.add_contacts(force_group=4)
protein.add_elec_switch(salt_conc, temperature, force_group=5)
protein.save_system('rigid_hp1alpha_dimer_CA_system.xml')
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBFile('hp1alpha_dimer_CA.pdb').getPositions()
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_dimer.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)